# 0 을 완료하기 위해서는 1을 끝내야 한다는 것을 [0,1] 쌍으로 표현하는 n 개의 코스가 있다. 
# 코스 개수 n 과 이 쌍들을 입력으로 받았을 때 모든 코스가 완료 가능한지 판별하라.

- 입력
```python
2, [[1,0]]
```
- 출력
```python
true
```

- 설명
    - 2개의 코스가 있으며, 1을 완료하기 위해 0을 끝내면 된다. 따라서 가능하다.

- 입력

```python
2, [[1,0],[0,1]]
```

```python
false
```
- 설명
    - 2개의 코스가 있으며, 1을 완료하기 위해 0을 끝내야 하고 0을 끝내기 위해서는 1을 끝내야 한다. 따라서 불가능하다.


In [ ]:
# DFS 로 순환 구조 판별하기


In [ ]:
# 가지치기로 최적화하기